In [1]:
import numpy as np
from systems import InvertedPendulum, CartPole
from evaluator import test_random_trajectories, test_single_trajectory, generate_lqr_dataset
from lqr import LQRExpert

In [2]:
# Create systems
pendulum = InvertedPendulum(dt=0.02)
cartpole = CartPole(dt=0.02)

# Create LQR controllers
lqr_pend = LQRExpert(pendulum)
lqr_cart = LQRExpert(cartpole)

print("Safe regions: ")
print(f"pend: {pendulum.safe_region}")
print(f"cart: {cartpole.safe_region}")

Safe regions: 
pend: {'theta': [-0.5, 0.5], 'theta_dot': [-1.0, 1.0]}
cart: {'x': [-2.4, 2.4], 'x_dot': [-5.0, 5.0], 'theta': [-0.209, 0.209], 'theta_dot': [-5.0, 5.0]}


#### Pendulum LQR Tests

In [3]:
print("Testing Pendulum with LQR...")

pend_results_verb = []
pend_results = {}

# Test 1: Small perturbation
success, final, max_state, traj = test_single_trajectory(lqr_pend, pendulum, np.array([0.1, 0.0]))
pend_results_verb.append(("Small Perturbation (0.1)", success, final))
pend_results["small"] = (success, final, max_state, traj)

# Test 2: Medium perturbation
success, final, max_state, traj = test_single_trajectory(lqr_pend, pendulum, np.array([0.3, 0.0]))
pend_results_verb.append(("Medium Perturbation (0.3)", success, final))
pend_results["medium"] = (success, final, max_state, traj)

# Test 3: Large perturbation (near boundary)
success, final, max_state, traj = test_single_trajectory(lqr_pend, pendulum, np.array([0.45, 0.0]))
pend_results_verb.append(("Large Perturbation (0.45)", success, final))
pend_results["large"] = (success, final, max_state, traj)

# Test 4: With initial velocity
success, final, max_state, traj = test_single_trajectory(lqr_pend, pendulum, np.array([0.2, 0.5]))
pend_results_verb.append(("Velocity (0.2, 0.5)", success, final))
pend_results["velocity"] = (success, final, max_state, traj)

# Test 5: Negative angle
success, final, max_state, traj = test_single_trajectory(lqr_pend, pendulum, np.array([-0.3, 0.0]))
pend_results_verb.append(("Negative (-0.3)", success, final))
pend_results["negative"] = (success, final, max_state, traj)

for i, (test, success, final_state) in enumerate(pend_results_verb):
    print(f"\nTest {i+1}: {test}")
    print(f"Safe = {success}, Final state = {final_state}")

Testing Pendulum with LQR...

Test 1: Small Perturbation (0.1)
Safe = True, Final state = [ 0.00048958 -0.0015133 ]

Test 2: Medium Perturbation (0.3)
Safe = True, Final state = [ 0.00221686 -0.00682271]

Test 3: Large Perturbation (0.45)
Safe = False, Final state = [2.98651742 3.6752525 ]

Test 4: Velocity (0.2, 0.5)
Safe = True, Final state = [ 0.00441103 -0.01344047]

Test 5: Negative (-0.3)
Safe = True, Final state = [-0.00221686  0.00682271]


In [4]:
success_rate = test_random_trajectories(lqr_pend, pendulum, n_trials=500, n_steps=200, verbose=True)

  Trials: 500
  Successes: 439/500 (87.8%)
  Failures: 61/500


In [5]:
states, actions = generate_lqr_dataset(pendulum, lqr_pend, 10000, save_path="./data/lqr_pend_data.pkl")

Generated dataset with 2000000 samples
State shape: (2000000, 2)
Action shape: (2000000, 1)
Saved to ./data/lqr_pend_data.pkl


#### Cartpole LQR Tests

In [5]:
cart_results_verb = []
cart_results = {}

# Test 1: Small angle perturbation
success, final, max_state, traj = test_single_trajectory(lqr_cart, cartpole, np.array([0.0, 0.0, 0.05, 0.0]))
cart_results_verb.append(("Small Angle Perturbation (0.05)", success, final))
cart_results["small_angle"] = (success, final, max_state, traj)

# Test 2: Medium angle perturbation
success, final, max_state, traj = test_single_trajectory(lqr_cart, cartpole, np.array([0.0, 0.0, 0.15, 0.0]))
cart_results_verb.append(("Medium Angle Perturbation (0.15)", success, final))
cart_results["medium_angle"] = (success, final, max_state, traj)

# Test 3: Large angle perturbation (near boundary)
success, final, max_state, traj = test_single_trajectory(lqr_cart, cartpole, np.array([0.0, 0.0, 0.2, 0.0]))
cart_results_verb.append(("Large Angle Perturbation (0.2)", success, final))
cart_results["large_angle"] = (success, final, max_state, traj)

# Test 4: Cart position perturbation
success, final, max_state, traj = test_single_trajectory(lqr_cart, cartpole, np.array([1.5, 0.0, 0.0, 0.0]))
cart_results_verb.append(("Cart Position Perturbation (1.5, 0)", success, final))
cart_results["cart_position"] = (success, final, max_state, traj)

# Test 5: Combined perturbations (cart + angle)
success, final, max_state, traj = test_single_trajectory(lqr_cart, cartpole, np.array([1.0, 0.0, 0.1, 0.0]))
cart_results_verb.append(("Combined Perturbation (1.0, 0, 0.1, 0)", success, final))
cart_results["combined"] = (success, final, max_state, traj)

# Test 6: With initial velocities
success, final, max_state, traj = test_single_trajectory(lqr_cart, cartpole, np.array([0.5, 1.0, 0.1, 0.5]))
cart_results_verb.append(("With Velocities (0.5, 1.0, 0.1, 0.5)", success, final))
cart_results["velocities"] = (success, final, max_state, traj)

# Test 7: Negative angle
success, final, max_state, traj = test_single_trajectory(lqr_cart, cartpole, np.array([0.0, 0.0, -0.15, 0.0]))
cart_results_verb.append(("Negative Angle (-0.15)", success, final))
cart_results["negative_angle"] = (success, final, max_state, traj)

# Test 8: Negative cart position
success, final, max_state, traj = test_single_trajectory(lqr_cart, cartpole, np.array([-1.5, 0.0, 0.0, 0.0]))
cart_results_verb.append(("Negative Cart Position (-1.5)", success, final))
cart_results["negative_position"] = (success, final, max_state, traj)

for i, (test, success, final_state) in enumerate(cart_results_verb):
    print(f"\nTest {i+1}: {test}")
    print(f"Safe = {success}, Final state = {final_state}")


Test 1: Small Angle Perturbation (0.05)
Safe = True, Final state = [ 0.053811   -0.03919991 -0.00100145  0.00500996]

Test 2: Medium Angle Perturbation (0.15)
Safe = True, Final state = [ 0.18354591 -0.12996615 -0.00387315  0.01712824]

Test 3: Large Angle Perturbation (0.2)
Safe = True, Final state = [ 0.26428755 -0.18515474 -0.00594701  0.02645886]

Test 4: Cart Position Perturbation (1.5, 0)
Safe = True, Final state = [ 0.4356307  -0.63673299  0.0316788   0.02655561]

Test 5: Combined Perturbation (1.0, 0, 0.1, 0)
Safe = True, Final state = [ 0.42057352 -0.51520191  0.01820651  0.02970524]

Test 6: With Velocities (0.5, 1.0, 0.1, 0.5)
Safe = False, Final state = [ 0.99799206 -0.78694861 -0.01141487  0.09381765]

Test 7: Negative Angle (-0.15)
Safe = True, Final state = [-0.18354591  0.12996615  0.00387315 -0.01712824]

Test 8: Negative Cart Position (-1.5)
Safe = True, Final state = [-0.4356307   0.63673299 -0.0316788  -0.02655561]


In [8]:
success_rate = test_random_trajectories(lqr_cart, cartpole, n_trials=500, n_steps=200, verbose=True)

  Trials: 500
  Successes: 134/500 (26.8%)
  Failures: 366/500
